In [9]:
# Importando os pacotes necessários para análise da base de dados 'insurance'

import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [10]:
# Carregando a base de dados do arquivo xlsx
dados = pd.read_excel('C:/Users/mclar/Downloads/Insurance.xlsx')

# 1. Avaliando as Estatísticas e Correlações
print(dados.describe())  # Estatísticas descritivas
print(dados.corr())  # Correlações entre as variáveis

# Análise por sexo
quantidade_sex_1 = dados[dados['sex'] == 1].shape[0]
print("Quantidade de indivíduos do sexo 1:", quantidade_sex_1)
medias_por_sexo = dados.groupby('sex').agg({'children': 'mean', 'expenses': 'mean'})
print(medias_por_sexo)

               age          sex          bmi     children       smoker  \
count  1338.000000  1338.000000  1338.000000  1338.000000  1338.000000   
mean     39.207025     0.494768    30.665471     1.094918     0.204783   
std      14.049960     0.500160     6.098382     1.205493     0.403694   
min      18.000000     0.000000    16.000000     0.000000     0.000000   
25%      27.000000     0.000000    26.300000     0.000000     0.000000   
50%      39.000000     0.000000    30.400000     1.000000     0.000000   
75%      51.000000     1.000000    34.700000     2.000000     0.000000   
max      64.000000     1.000000    53.100000     5.000000     1.000000   

           expenses    northeast    northwest    southeast    southwest  
count   1338.000000  1338.000000  1338.000000  1338.000000  1338.000000  
mean   13270.422414     0.242152     0.242900     0.272048     0.242900  
std    12110.011240     0.428546     0.428995     0.445181     0.428995  
min     1121.870000     0.000000     

In [11]:
# 2. Selecionando as Variáveis (supondo que age, bmi, smoker, e children são as variáveis explicativas)
X = dados[['age', 'sex', 'bmi', 'smoker', 'children', 'northeast', 'northwest', 'southeast', 'southwest' ]]  # Variáveis explicativas
y = dados['expenses']  # Variável dependente

# 3. Estimando o Modelo de Regressão
X = sm.add_constant(X)  # Adicionando uma coluna de uns para o termo constante
modelo = sm.OLS(y, X).fit()  # Estimando o modelo de regressão

# 4. Análise de Diagnóstico do Modelo
print(modelo.summary())  # Resumo do modelo para análise de diagnóstico

                            OLS Regression Results                            
Dep. Variable:               expenses   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.749
Method:                 Least Squares   F-statistic:                     500.9
Date:                Mon, 29 Apr 2024   Prob (F-statistic):               0.00
Time:                        12:16:33   Log-Likelihood:                -13548.
No. Observations:                1338   AIC:                         2.711e+04
Df Residuals:                    1329   BIC:                         2.716e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.013e+04    791.551    -12.795      0.0

In [12]:
# 5. Interpretando os Coeficientes Estimadospor OLS
print(modelo.params)  # Coeficientes estimados do modelo

const       -10127.869905
age            256.839171
sex            131.352014
bmi            339.289863
smoker       23847.476695
children       475.688916
northeast    -1945.044570
northwest    -2297.834665
southeast    -2980.640271
southwest    -2904.350399
dtype: float64


In [14]:
# 6. Modelo com Regularização (Ridge Regression)
# Dividindo os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar o modelo de Ridge Regression
ridge_model = Ridge(alpha=1.0)  
ridge_model.fit(X_train, y_train)

# Fazendo previsões com o conjunto de teste
y_pred = ridge_model.predict(X_test)

# Calculando as métricas de desempenho do modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Imprimindo as métricas de desempenho
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R²): {r2}")

# Imprimindo os coeficientes do modelo
print("Coeficientes do Modelo:")
for feature, coef in zip(X.columns, ridge_model.coef_):
    print(f"{feature}: {coef}")

Mean Squared Error (MSE): 33646045.77642958
R-squared (R²): 0.7832765204024632
Coeficientes do Modelo:
const: 0.0
age: 256.7624766134751
sex: 10.641432527164723
bmi: 337.13480362050865
smoker: 23514.207456744865
children: 426.17413565777224
northeast: 458.7047975994694
northwest: 85.78620561927535
southeast: -193.26471997307402
southwest: -351.22628324666266


In [15]:
f_test = modelo.f_test(ridge_model.coef_)
print("Teste F para o modelo com regularização (Ridge Regression):")
print(f_test)

Teste F para o modelo com regularização (Ridge Regression):
<F test: F=3340.3349364488886, p=0.0, df_denom=1.33e+03, df_num=1>
